# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Mentoría 16 - ¿Cómo identificar fuga de ventas? Inteligencia artificial aplicada al sector comercial.

### Explorando Patrones de Datos a través de Clustering (TP3) - Parte 2: Análisis e imputación de NaNs

**Integrantes:**
- Canalis, Patricio.
- Chevallier-Boutell, Ignacio José.
- Villarroel Torrez, Daniel.

**Mentores:**
- Gonzalez, Lucía
- Lahoz, Nahuel

---
## Librerías

In [ ]:
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Preferencias
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

---
# Identificación de datos faltantes

## Sin distinguir por subrubro

Vemos que hay una gran cantidad de datos faltantes en todas las variables asociadas a la variación porcentual. Más aún: ninguna de estas columnas posee el 100% de los datos. Para cada grupo de variables (según sea comisión/ventas o anual/cuatrimestral) la cantidad de datos faltantes disminuye al progresar en el tiempo (lo cual tiene sentido).

Las variables asociadas a la comisión presentan más datos faltantes que aquellas asociadas a las ventas. Dentro de ventas, hay más datos en el caso cuatrimestral que en el caso anual.

**Nota:** En la gráfica aparecen, desde arriba hacia abajo, ventas anuales, comisión anuales, ventas cuatrimestrales y comisión cuatrimestrales. Dentro de cada grupo, aparecen desde el mes más reciente hacia el más antigüo.

In [ ]:
msno.bar(tp3.iloc[:, 3:], fontsize=12, color="tab:green", figsize=(24, 20))
plt.show()

In [ ]:
msno.bar(tp3[['F_pct_Ven_1905', 'F_pct_Ven_2206', 'F_pct_Com_1905', 'F_pct_Com_2206', 'Y_pct_Ven_2001', 'Y_pct_Ven_2206', 'Y_pct_Com_2001', 'Y_pct_Com_2206']], fontsize=12, color="tab:green", figsize=(12, 10))
plt.ylim(0.6, 1)
plt.show()

Por lo antes dicho, seguiremos el análisis con la ventas intercuatrimestrales.

In [ ]:
basics = ['ID', 'Subrubro', 'Modelo']

F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
         'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
         'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
         'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
         'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
         'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
         'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
         'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
         'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
         'F_pct_Ven_2205', 'F_pct_Ven_2206']

tp3_Fven = tp3[basics+F_Ven].copy()

In [ ]:
msno.bar(tp3_Fven, fontsize=12, color="tab:green", figsize=(12, 10))
plt.ylim(0.8, 1)
plt.show()

## Diferenciando por subrubro

Hay de todo un poco. Algunos casos presentan menos de 80% de los datos. La mayoría sigue la tendencia de ir aumentando a medida que pasa el tiempo. Algunos son más "consistentes" que otros.

In [ ]:
for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[tp3_Fven['Subrubro'] == sr].copy()

    plt.suptitle(sr)
    msno.bar(tp3sr, fontsize=12, color="tab:green", figsize=(10, 7))
    plt.ylim(0.8, 1)
    plt.show()

## Cantidad de vacíos por vendedor

De las 41 columnas, 3 son el `ID`, el `Subrubro` y `Modelo`. Las otras 38 corresponder a las variaciones porcentuales intercuatrimestrales. Si tomamos como aceptable un 10% de datos faltantes, podemos tolerar como máximo que haya 4 datos faltantes.

De los 4544 vendedores únicos, 3488 (el 77 %) no presentan ningún dato faltante. En el otro extremo, llegamos a tener vendedores con 37 datos faltantes, siendo que son 38 columnas. Acá hay algo raro, que debemos analizar.

In [ ]:
# Paso 1: Contar valores vacíos por fila
vacios_por_fila = tp3_Fven.isnull().sum(axis=1)

# Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
resumen = vacios_por_fila.value_counts().reset_index()
resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')

plt.figure(figsize=(10, 4))  # Tamaño del gráfico
plt.bar(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], color='blue', alpha=0.7)
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor')
plt.xticks(resumen['Cantidad de Valores Vacíos'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, 200)

plt.show()

display(resumen)

Analicemos esto mismo, pero por subrubro. `Miscelaneo` presenta la mayor cantidad de vendedores con diferente cantidad de datos faltantes. En todos los casos hay algún vendedor con más de 30 datos faltantes.

In [ ]:
plt.figure(figsize=(10, 4))  # Tamaño del gráfico

for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[tp3_Fven['Subrubro'] == sr].copy()

    # Paso 1: Contar valores vacíos por fila
    vacios_por_fila = tp3sr.isnull().sum(axis=1)

    # Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
    resumen = vacios_por_fila.value_counts().reset_index()
    resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
    resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')

    plt.plot(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], label=sr, marker='o')
    
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor, según subrubro')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(0, 30)
plt.xlim(0, 40)

plt.legend(fontsize=7)

plt.show()

Vemos que en todos los subrubros tenemos modelo (por las dudas si nos había quedado alguno sin).

In [ ]:
print('Subrubro \t Cantidad de modelos')
for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[tp3_Fven['Subrubro'] == sr].copy()
    
    print(f"{sr} \t {tp3sr['Modelo'].sum()}")

Vamos a repetir la gráfica de valores faltantes por subrubro, pero ahora tomando únicamente a los modelo. Sólo se grafican aquellos subrubros donde algún modelo posee al menos 1 dato  faltante. Estos subrubros resultaron ser: `Comb.`, `Supermercados`, `Comb. Ley`, `Vehiculos` y `Tabaco`. Los últimos 4 casos presentan modelos con una cantidad de datos faltantes muy por encima del valor tolerable.

In [ ]:
for sr in tp2.Subrubro.unique():

    tp3sr = tp3_Fven[(tp3_Fven['Modelo'] == 1) & (tp3_Fven['Subrubro'] == sr)].copy()

    # Paso 1: Contar valores vacíos por fila
    vacios_por_fila = tp3sr.isnull().sum(axis=1)

    # Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
    resumen = vacios_por_fila.value_counts().reset_index()
    resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
    resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
    if len(resumen) > 1:
        plt.plot(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], label=sr, marker='o')
    
plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Vacíos por Vendedor, según subrubro')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xlim(-1, 40)

plt.legend(fontsize=7)

plt.show()

Vamos a identificar a estos modelos con datos faltantes y corroborar si no es que pertenecen a más de un subrubro. Esto explicaría el porque de que aparezcan con datos faltantes en algún rubro.

Tomamos las siguientes decisiones
+ Eliminar al vendedor 306 de `Supermercados` y `Tabaco`, ya que sólo tiene 2 datos al principio.
+ Los 12 NaN del vendedor 494 correspoden a que vendía sólo en `Comb.` y con el tiempo empezó a vender también en `Comb. Ley`. Por este motivo no se lo elimina, sino que se lo imputará de alguna manera.
+ El vendedor 496 presenta un único dato faltante en dos de tres subrubros. Se lo imputará de alguna manera.
+ El vendedor 599 presenta 3 datos faltantes en uno de sus subrubros. Se lo imputará de alguna manera.
+ El vendedor 798 presenta 27 datos faltantes en `Vehiculos` y entre los 11 valores que no son NaN, solo hay 4 que son no nulos. Se lo elimina de esta categoría.
+ El vendedor 891 presenta un único dato faltante en un subrubro. Se lo imputará de alguna manera.

In [ ]:
# Busco los ID que tienen datos faltantes
ids = []
for sr in ['Supermercados', 'Comb.', 'Comb. Ley', 'Vehiculos', 'Tabaco']:

    tp3sr = tp3_Fven[(tp3_Fven['Modelo'] == 1) & (tp3_Fven['Subrubro'] == sr)]['ID'].copy()
    for k in tp3sr:
        ids.append(k)

ids = np.unique(np.array(ids))

for k in ids:
    tp3sr = tp3_Fven[(tp3_Fven['ID'] == k)].copy()
    vacios_por_fila = tp3sr.isnull().sum(axis=1)
    resumen = vacios_por_fila.value_counts().reset_index()
    resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
    resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
    if len(resumen) > 1:
        print(f'\n\nVendedor {k}:')
        display(tp3sr)
        display(resumen)

# for sr in tp2.Subrubro.unique():

#     tp3sr = tp3_Fven[(tp3_Fven['Modelo'] == 1) & (tp3_Fven['Subrubro'] == sr)].copy()

#     # Paso 1: Contar valores vacíos por fila
#     vacios_por_fila = tp3sr.isnull().sum(axis=1)

#     # Paso 2: Generar un resumen de cuántas filas tienen 0, 1, 2, 3, etc., valores vacíos
#     resumen = vacios_por_fila.value_counts().reset_index()
#     resumen.columns = ['Cantidad de Valores Vacíos', 'Número de Filas']
#     resumen = resumen.sort_values(by='Cantidad de Valores Vacíos')
#     if len(resumen) > 1:
#         plt.plot(resumen['Cantidad de Valores Vacíos'], resumen['Número de Filas'], label=sr, marker='o')
    
# plt.xlabel('Cantidad de Valores Vacíos por Vendedor')
# plt.ylabel('Frecuencia')
# plt.title('Distribución de Valores Vacíos por Vendedor, según subrubro')
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.xlim(-1, 40)

# plt.legend(fontsize=7)

# plt.show()

In [ ]:
tp3_Fven[['ID']].value_counts()

In [ ]:
# msno.matrix(tp3.iloc[:, 3:], fontsize=12, color=[0.5,0,0], figsize=(6, 5))
# plt.show()

In [ ]:
# # Definir grupos de variables:

# basics = ['ID', 'Subrubro', 'Modelo']

# F_Com = ['F_pct_Com_1905', 'F_pct_Com_1906', 'F_pct_Com_1907', 'F_pct_Com_1908', 
#          'F_pct_Com_1909', 'F_pct_Com_1910', 'F_pct_Com_1911', 'F_pct_Com_1912',
#          'F_pct_Com_2001', 'F_pct_Com_2002', 'F_pct_Com_2003', 'F_pct_Com_2004',
#          'F_pct_Com_2005', 'F_pct_Com_2006', 'F_pct_Com_2007', 'F_pct_Com_2008',
#          'F_pct_Com_2009', 'F_pct_Com_2010', 'F_pct_Com_2011', 'F_pct_Com_2012',
#          'F_pct_Com_2101', 'F_pct_Com_2102', 'F_pct_Com_2103', 'F_pct_Com_2104',
#          'F_pct_Com_2105', 'F_pct_Com_2106', 'F_pct_Com_2107', 'F_pct_Com_2108',
#          'F_pct_Com_2109', 'F_pct_Com_2110', 'F_pct_Com_2111', 'F_pct_Com_2112',
#          'F_pct_Com_2201', 'F_pct_Com_2202', 'F_pct_Com_2203', 'F_pct_Com_2204',
#          'F_pct_Com_2205', 'F_pct_Com_2206']

# F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
#          'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
#          'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
#          'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
#          'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
#          'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
#          'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
#          'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
#          'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
#          'F_pct_Ven_2205', 'F_pct_Ven_2206']

# Y_Com = ['Y_pct_Com_2001', 'Y_pct_Com_2002', 'Y_pct_Com_2003', 'Y_pct_Com_2004',
#          'Y_pct_Com_2005', 'Y_pct_Com_2006', 'Y_pct_Com_2007', 'Y_pct_Com_2008',
#          'Y_pct_Com_2009', 'Y_pct_Com_2010', 'Y_pct_Com_2011', 'Y_pct_Com_2012',
#          'Y_pct_Com_2101', 'Y_pct_Com_2102', 'Y_pct_Com_2103', 'Y_pct_Com_2104',
#          'Y_pct_Com_2105', 'Y_pct_Com_2106', 'Y_pct_Com_2107', 'Y_pct_Com_2108',
#          'Y_pct_Com_2109', 'Y_pct_Com_2110', 'Y_pct_Com_2111', 'Y_pct_Com_2112',
#          'Y_pct_Com_2201', 'Y_pct_Com_2202', 'Y_pct_Com_2203', 'Y_pct_Com_2204',
#          'Y_pct_Com_2205', 'Y_pct_Com_2206']

# Y_Ven = ['Y_pct_Ven_2001', 'Y_pct_Ven_2002', 'Y_pct_Ven_2003', 'Y_pct_Ven_2004',
#          'Y_pct_Ven_2005', 'Y_pct_Ven_2006', 'Y_pct_Ven_2007', 'Y_pct_Ven_2008',
#          'Y_pct_Ven_2009', 'Y_pct_Ven_2010', 'Y_pct_Ven_2011', 'Y_pct_Ven_2012',
#          'Y_pct_Ven_2101', 'Y_pct_Ven_2102', 'Y_pct_Ven_2103', 'Y_pct_Ven_2104',
#          'Y_pct_Ven_2105', 'Y_pct_Ven_2106', 'Y_pct_Ven_2107', 'Y_pct_Ven_2108',
#          'Y_pct_Ven_2109', 'Y_pct_Ven_2110', 'Y_pct_Ven_2111', 'Y_pct_Ven_2112',
#          'Y_pct_Ven_2201', 'Y_pct_Ven_2202', 'Y_pct_Ven_2203', 'Y_pct_Ven_2204',
#          'Y_pct_Ven_2205', 'Y_pct_Ven_2206']

# Tratamiento de vacíos

In [ ]:
# Para que las funciones se actualicen sin tener que refrescar el kernel
%load_ext autoreload
%autoreload 2

# Funciones de visualización y curación
import pandas as pd
import json
from os.path import exists
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from scipy.stats import linregress as LR
from scipy.stats import skew, kurtosis, skewtest, kurtosistest

# Funciones de clustering
from sklearn.cluster import KMeans, MeanShift
from sklearn import manifold, preprocessing, decomposition

# Funciones propias
from tp2_utils_limpieza import * 

# Preferencias
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

In [ ]:
tp3 = pd.read_csv('../data/interim/tp3_vendedores_vector.csv')
tp3 = limpiar_basic(tp3, cols_drop='Omega')

basics = ['ID', 'Subrubro', 'Modelo']

F_Ven = ['F_pct_Ven_1905', 'F_pct_Ven_1906', 'F_pct_Ven_1907', 'F_pct_Ven_1908', 
         'F_pct_Ven_1909', 'F_pct_Ven_1910', 'F_pct_Ven_1911', 'F_pct_Ven_1912',
         'F_pct_Ven_2001', 'F_pct_Ven_2002', 'F_pct_Ven_2003', 'F_pct_Ven_2004',
         'F_pct_Ven_2005', 'F_pct_Ven_2006', 'F_pct_Ven_2007', 'F_pct_Ven_2008',
         'F_pct_Ven_2009', 'F_pct_Ven_2010', 'F_pct_Ven_2011', 'F_pct_Ven_2012',
         'F_pct_Ven_2101', 'F_pct_Ven_2102', 'F_pct_Ven_2103', 'F_pct_Ven_2104',
         'F_pct_Ven_2105', 'F_pct_Ven_2106', 'F_pct_Ven_2107', 'F_pct_Ven_2108',
         'F_pct_Ven_2109', 'F_pct_Ven_2110', 'F_pct_Ven_2111', 'F_pct_Ven_2112',
         'F_pct_Ven_2201', 'F_pct_Ven_2202', 'F_pct_Ven_2203', 'F_pct_Ven_2204',
         'F_pct_Ven_2205', 'F_pct_Ven_2206']

tp3_Fven = tp3[basics+F_Ven].copy()

tp3_Fven['Vacios'] = tp3_Fven[F_Ven].isnull().sum(axis=1)

tp3_Fven

En primer lugar, vamos a definir una variable llamada `Tipo_subrubro` que determina si el rubro en cuestión es *Primario* o *Secundario*. Donde cada vendedor va a tener un único rubro *Primario* y ninguno, uno o más rubros *Secundario*.
* Los vendedores con un solo rubro, van a tener el valor *Primario* en todos los casos.
* Para los vendedores con varios rubros, se tendrá en cuenta la siguiente regla de decisión:
    * Es *Primario* el subrubro con menor cantidad de valores vacíos (en caso de empate define el que tenga el mayor volumen de ventas)
    * El resto de los subrubros son *Secundario*.

In [ ]:
# Encuentra los ID que aparecen más de una vez
duplicados = tp3_Fven[tp3_Fven.duplicated(subset='ID', keep=False)].sort_values(by=['ID', 'Subrubro']).copy()
duplicados

Necesitamos recuperar la variable `Ventas`

In [ ]:
path = '../data/interim/tp3_registros_vendedores_abs.csv'
montos = pd.read_csv(path)
montos = montos.sort_values(by=['ID', 'Subrubro']).copy()
montos

In [ ]:
# Crear la nueva variable 'Montos' que contendrá la suma de las Ventas por Subrubro e ID
montos['Montos'] = montos.groupby(['ID', 'Subrubro'])['Ventas'].transform('sum')
montos = montos[['ID', 'Subrubro', 'Montos']].copy()
montos = montos.drop_duplicates(subset=['ID', 'Subrubro'], keep='first').copy()
montos

In [ ]:
# Mergear la variable Montos al df duplicados
duplicados = duplicados.merge(montos[['ID', 'Subrubro', 'Montos']], on=['ID', 'Subrubro'], how='left')
duplicados

In [ ]:
def etiquetar_tipo_subrubro(df_grupo):
    # Si hay empate en la cantidad de Vacíos, seleccionamos el que tenga el menor Promedio
    min_vacios = df_grupo['Vacios'].min()
    max_montos = df_grupo[df_grupo['Vacios'] == min_vacios]['Montos'].max()
    
    # Identificar el Primario y Secundarios
    df_grupo['Tipo_subrubro'] = 'Secundario'
    df_grupo.loc[(df_grupo['Vacios'] == min_vacios) & (df_grupo['Montos'] == max_montos), 'Tipo_subrubro'] = 'Primario'
    
    return df_grupo

# Aplicar la función de etiquetado por grupo (ID)
duplicados = duplicados.groupby('ID').apply(etiquetar_tipo_subrubro)
duplicados.reset_index(drop=True, inplace=True)

duplicados

In [ ]:
duplicados = duplicados[['ID', 'Subrubro', 'Tipo_subrubro']].copy()
duplicados

In [ ]:
# Mergear la variable Tipo_subrubro al df tp3_Fven
tp3_Fven = tp3_Fven.merge(duplicados[['ID', 'Subrubro', 'Tipo_subrubro']], on=['ID', 'Subrubro'], how='left')

# Para los vacíos, asignar Primario
tp3_Fven['Tipo_subrubro'].fillna("Primario", inplace=True)

tp3_Fven

Una vez definida la variable `Tipo_subrubro`, vamos a tomar determinaciones respecto a qué hacer con los valores nulos.
Las reglas de decisión van a tener en cuenta a las variables `Modelo`, `Tipo_subrubro` y `Vacios`.

| Modelo | Tipo_subrubro | Vacíos         | Decisión       |
|--------|---------------|----------------|----------------|
| 1      | Primario      | 0              | No hacer nada  |
| 1      | Primario      | Hasta 20 (50%) | Imputar        |
| 1      | Primario      | Más de 20      | Tirar          |
| 1      | Secundario    | 0              | No hacer nada  |
| 1      | Secundario    | Hasta 10 (25%) | Imputar        |
| 1      | Secundario    | Más de 10      | Tirar          |
| 0      | Primario      | 0              | No hacer nada  |
| 0      | Primario      | Hasta 4 (10%)  | Imputar        |
| 0      | Primario      | Más de 4       | Tirar          |
| 0      | Secundario    | 0              | No hacer nada  |
| 0      | Secundario    | Hasta 2 (5%)   | Imputar        |
| 0      | Secundario    | Más de 2      | Tirar          |

In [ ]:
# Crear una función para determinar la decisión
def determinar_decision(row):
    if row['Modelo'] == 1:
        if row['Tipo_subrubro'] == 'Primario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 20:
                return 'Imputar'
            else:
                return 'Tirar'
        elif row['Tipo_subrubro'] == 'Secundario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 10:
                return 'Imputar'
            else:
                return 'Tirar'
    elif row['Modelo'] == 0:
        if row['Tipo_subrubro'] == 'Primario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 4:
                return 'Imputar'
            else:
                return 'Tirar'
        elif row['Tipo_subrubro'] == 'Secundario':
            if row['Vacios'] == 0:
                return 'No hacer nada'
            elif row['Vacios'] <= 2:
                return 'Imputar'
            else:
                return 'Tirar'
    return 'Desconocido'

In [ ]:
# Aplicar la función para crear la columna "Decisión"
tp3_Fven['Decision'] = tp3_Fven.apply(determinar_decision, axis=1)
tp3_Fven

## Aplicación de las decisiones

#### Tirar

In [ ]:
tp3_Fven_limpio = tp3_Fven[tp3_Fven['Decision'] != 'Tirar'].copy()
tp3_Fven_limpio.drop(columns=['Decision', 'Vacios', 'Tipo_subrubro'], inplace=True)
tp3_Fven_limpio

In [ ]:
tp3_Fven_limpio.to_csv('../data/interim/base.csv', index=False)

### Imputar con ceros

Para empezar, imputamos por 0 (luego decidir qué hacer)

In [ ]:
# Imputar los valores faltantes con 0 en las columnas especificadas
tp3_Fven_limpio_ceros = tp3_Fven_limpio.copy()
tp3_Fven_limpio_ceros[F_Ven] = tp3_Fven_limpio_ceros[F_Ven].fillna(0).copy()
tp3_Fven_limpio_ceros

### Comparación de datasets antes y después del tratamiento de nulos

In [ ]:
# Crear los DataFrames "antes" y "ahora"
antes = tp3_Fven.pivot_table(index='Subrubro', columns='Modelo', aggfunc='size', fill_value=0)
ahora = tp3_Fven_limpio_ceros.pivot_table(index='Subrubro', columns='Modelo', aggfunc='size', fill_value=0)
diferencia = ahora - antes
porcentual = (ahora - antes)/antes*100

# Agregar nombres a los DataFrames
antes = antes.rename(columns=lambda x: (x, 'antes'))
ahora = ahora.rename(columns=lambda x: (x, 'después'))
diferencia = diferencia.rename(columns=lambda x: (x, 'diferencia'))
porcentual = porcentual.rename(columns=lambda x: (x, 'porcentual'))

# Concatenar los DataFrames con nombres
resultado = pd.concat([antes, ahora, diferencia, porcentual], axis=1)
resultado.columns = pd.MultiIndex.from_tuples(resultado.columns)
resultado.columns = resultado.columns.swaplevel(0, 1)  # Intercambiar el orden de los niveles de las columnas
resultado.columns.names = ['Tabla', 'Modelo']  # Cambiar el orden de los nombres de los niveles

# Mostrar el resultado
resultado

In [ ]:
tp3_Fven_limpio_ceros.to_csv('../data/interim/tp3_Fven_limpio_ceros.csv', index=False)

#### Imputar con KNN

In [ ]:
tp3_Fven_limpio.columns

In [ ]:
# transponemos para imputar, porque knn funciona asignando el valor en función de las otras columnas. 
# Acá como cada feature en el mismo usuario en otro momento del tiempo los transpuse para que las columnas 
# sean otros usuarios en el mismo momento del tiempo.


X = tp3_Fven_limpio.iloc[:, 2:41].transpose() 
X.head(10)

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
imputer.fit(X)
X_imputed_arr = imputer.transform(X)

X_imputed_df_trans = pd.DataFrame(X_imputed_arr, columns=X.columns)

# vuelve a transponer para volver a la forma original
X_imputed_df = X_imputed_df_trans.transpose()
X_imputed_df.head()

In [ ]:
X_imputed = tp3_Fven_limpio.copy()
# inserto el df imputado en la misma forma de df del que se partió para tener las otras features categóricas (Subrubro, principalmente)
X_imputed.iloc[:, 2:41] = X_imputed_df

X_imputed.tail(10)